In [189]:
q = 2 
m = 6 
p = x^6 + x^4 + x^3 + x + 1
F.<a> = GF(q^m, modulus=p)
R.<x> = PolynomialRing(F) 

Suppose 
    m = 6 </br>
    a is prim. elem  </br>
    q = 2  </br>
    p(a) = a^6 + a^4 + a^3 + a + 1 


In [190]:
g = x^7 + x + 1
g_t = [1, 0, 0, 0, 0, 0, 1] # hardcoded for now: 
g_t

[1, 0, 0, 0, 0, 0, 1]

In [233]:
n = q^m 
T = g.degree() 
T

7

In [276]:
L =[a^i for i in range(1, n + 1) if g(a^i) != 0 ]
len(L)

64

In [277]:
# lookup_alpha
# L =[a^i for i in range(1, n+1) if g(a^i) != 0 ]
print(len(L))
Z = matrix(F, [[(1 / g(el)) if i == j else 0 for i, el in enumerate(L)] for j in range(n)])
Y = matrix(F, [[a_i^t for a_i in L] for t in range(T)])
X = [[0] * len(g_t) for _ in range(len(g_t))]
for i in range(len(g_t)): 
    for j in range(len(g_t)): 
        if j >= i: 
            X[j][i] = g_t[j-i]
X = matrix(F, X) 
print(f"dimension Z {Z.dimensions()}")
print(f"dimension Y {Y.dimensions()}")
print(f"dimension X {X.dimensions()}")

64
dimension Z (64, 64)
dimension Y (7, 64)
dimension X (7, 7)


In [278]:
H = X*Y*Z.T
H.dimensions()
H_ = [] 
for row in H.T: 
    vec = vector(F, list([vector(i) for i in row]))
    sss = [] 
    for r in vec: 
        sss += r 
    H_.append(sss) 
H_ = matrix(H_).T
H_.dimensions()

(42, 64)

In [279]:
generator_matrix = H_.right_kernel_matrix()
null_basis = generator_matrix*H.T 
if any(null_basis): 
    print("failed") 
else:
    print("worked!") 
generator_matrix.dimensions()

worked!


(22, 64)

Sanity check:

In [280]:
C = codes.GoppaCode(g, L) 
print(f"{C.minimum_distance()}") 
print(f"{C.distance_bound()}")
print(f"{C.generator_matrix().dimensions()}")
HHH = C.parity_check_matrix()
HHH.row_space() == H_.row_space()


2
8
(22, 64)


True

In [281]:
text = "TIMEFLIESQUICKLY" 

def text2bin(msg):
    return "".join(f"{ord(i):08b}" for i in msg) 
def bin2text(bin_s): 
    return "".join([chr(int(bin_s[i:i+8], 2)) for i in range(0, len(bin_s), 8)])
def to_field_element(string):
    """Binary String to field Element"""
    elem = 0
    n = len(string)
    for i in range(n):
        coeff = int(string[n-i-1])
        elem += coeff * a^i # coeff is 0 or 1
    return elem

def encodeGoppa(m):
    bin_m = text2bin(m) 
    m = m + "0"*(128 - len(bin_m)) 
    bin_m_chunks = [bin_m[i:i+64] for i in range(0, len(bin_m), 64)]
    half_blocks = []
    for half in bin_m_chunks:
        r_i = 0 
        for i, value in enumerate(half): 
            r_i += int(value) / (x - (a^i)) 
        if r_i.mod(g) == 0: # 
            half_blocks.append(r_i)
        else: 
            raise Exception("Was not zero!")
        r_i = 0 
    return half_blocks


code_word = encodeGoppa(text)

In [283]:
generator_matrix

22 x 64 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

# Goppa decoding 

Pattersons algortihm for decoding. Corrects t design errors.  
Patterson converts a syndrome to a vector of erros. The syndrome of a binary word c = (c_0, ..., c_n-1) is 

In [302]:
## import random 
# r = vector(F, (0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1))
r = vector(GF(2), [1 for _ in range(22)])

r = vector(F, generator_matrix.T * r)
# rr = vector(GF(2), valid_r * generator_matrix.T)
#c = generator_matrix * r
r

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0)

In [320]:
h = prod((x-value) for value in L)
h

x^64 + a*x^63 + x + a

In [ ]:
# input to goppa decoding is a binary vector, of vector in base field 
def find_syndromes(received_vector):  
    codelocators = [] 
    F_2m = g.base_ring()
    Z = F_2m.gen()
    for i in range(2^m - 1): 
        codelocators.append(Z^(i+1))
    print(F_2m(0))
    codelocators.append(F_2m(0)) 
    PR_f_2m = g.parent() 
    X = PR_f_2m.gen() 
    syndrome_calc = matrix(PR_f_2m, 1, len(codelocators)) 
    for i in range(len(codelocators)): 
        syndrome_calc[0, i] = (x -  codelocators[i]).inverse_mod(g) 
    return syndrome_calc * received_vector
    
    
def find_syndromesV3(received_vector): 
    F_2m = received_vector.base_ring()
    codelocators = [F_2m(x + 1) for x in range(F_2m.degree())]
    codelocators.append(F_2m.zero)
    syndrome_calc = matrix([[(F_2m(x) - c).inverse_mod(received_vector.parent().base_ring())
                             for c in codelocators]
                            for x in F_2m])
    return syndrome_calc * received_vector

def GoppaDecoding(s_x, g_x):
    
    pass 

find_syndromesV2(r) 


In [305]:
# r = vector(F, (1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0))

Finite Field in a of size 2^6


In [307]:
syndrome_calc * r

(0)